## PLST - Phuture Liquid Staking Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0
liveness_threshold = 30 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["liquid-staking-tokens","liquid-restaking-tokens"]

# Calculating max slippage based on liquidity threshold
liq = 1.5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results, liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    remove_category_assets=['frax-ether'],
    values_to_update=[{'id':'benqi-liquid-staked-avax','category':'symbol','value':"sAVAX"},
                      {'id':'rocket-pool-eth','category':'symbol','value':"rETH"},
                      {'id':'staked-ether','category':'symbol','value':"stETH"},
                      {'id':'staked-frax-ether','category':'symbol','value':"sfrxETH"},
                      {'id':'stader-ethx','category':'symbol','value':"ETHx"},
                      {'id':'stakewise-v3-oseth','category':'symbol','value':"osETH"}
                      ],
    onchain_oracles=['coinbase-wrapped-staked-eth']
    )
liquidity

/Users/olivermehr/Documents/GitHub/rebalancing-data/PLST/../methodology.py:135: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.category_data = pd.concat(dataframe_list, axis=0)


Note: reth, marketcap data available only for 0 < 30 days
Note: safestake, marketcap data available only for 0 < 30 days
Note: ankr-reward-bearing-ftm, marketcap data available only for 0 < 30 days
Note: ankr-reward-earning-matic, marketcap data available only for 0 < 30 days
Note: ankr-staked-bnb, marketcap data available only for 0 < 30 days
Note: binance-eth, marketcap data available only for 0 < 30 days
Note: kelp-dao-restaked-eth, marketcap data available only for 0 < 30 days
Note: lido-staked-matic, marketcap data available only for 0 < 30 days
Note: liquid-staked-ethereum, marketcap data available only for 0 < 30 days
Note: pstake-staked-bnb, marketcap data available only for 0 < 30 days
0xbc011a12da28e8f0f528d9ee5e7039e22f91cf18 arbitrum-one
0x3cd55356433c89e50dc51ab07ee0fa0a95623d53 binance-smart-chain
0x6d1fdbb266fcc09a16a22016369210a15bb95761 polygon-pos
0xb90ccd563918ff900928dc529aa01046795ccb4a fantom
0xf03a7eb46d01d9ecaa104558c732cf82f6b6b645 ethereum
0x12d8ce035c5de3ce39

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,fantom,optimistic-ethereum,polygon-pos,best slippage,best slippage chain,verify homechain slippage
id,,,,,,,,,,,
staked-ether,NaN,NaN,NaN,NaN,-0.002799,NaN,NaN,NaN,-0.002799,ethereum,False
rocket-pool-eth,-0.001830,NaN,-0.006503,NaN,-0.002348,NaN,-0.007523,-0.966564,-0.001830,arbitrum-one,True
mantle-staked-ether,NaN,NaN,NaN,NaN,-0.003885,NaN,NaN,NaN,-0.003885,ethereum,False
sweth,NaN,NaN,NaN,NaN,-0.004727,NaN,NaN,NaN,-0.004727,ethereum,False
coinbase-wrapped-staked-eth,-0.007874,NaN,-0.009177,NaN,-0.004172,NaN,-0.979224,-0.766896,-0.004172,ethereum,True
stader-ethx,NaN,NaN,NaN,NaN,-0.924112,NaN,NaN,NaN,-0.924112,ethereum,False
benqi-liquid-staked-avax,NaN,-0.005858,NaN,NaN,NaN,NaN,NaN,NaN,-0.005858,avalanche,False
stader-maticx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.005461,-0.005461,polygon-pos,False
seth2,NaN,NaN,NaN,NaN,-0.005509,NaN,NaN,NaN,-0.005509,ethereum,False


In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
staked-ether,stETH,3.478132e+10,0.300000,19661,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,ethereum
rocket-pool-eth,rETH,2.151076e+09,0.248984,16318,0xec70dcb4a1efa46b8f2d97c310c9c4790ba5ffa8,arbitrum-one
mantle-staked-ether,METH,1.893684e+09,0.219191,14365,0xd5f7838f5c461feff7fe49ea5ebaf7728bb0adfa,ethereum
sweth,SWETH,8.770783e+08,0.101520,6654,0xf951e335afb289353dc249e82926178eac7ded78,ethereum
coinbase-wrapped-staked-eth,CBETH,7.022488e+08,0.081284,5326,0x1debd73e752beaf79865fd6446b0c970eae7732f,arbitrum-one
benqi-liquid-staked-avax,sAVAX,2.828694e+08,0.032742,2145,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,avalanche
ankreth,ANKRETH,7.806737e+07,0.009036,592,0xe05a08226c49b636acf99c40da8dc6af83ce5bb3,arbitrum-one
stakewise-v3-oseth,osETH,6.257605e+07,0.007243,474,0xf1c9acdc66974dfb6decb12aa385b9cd01190e38,ethereum
